In [1]:
import sagemaker
from sagemaker import model, ModelPackage
from sagemaker import predictor
import boto3
from datetime import datetime
from time import strftime, gmtime

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client('sagemaker')

client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]

In [3]:
from sagemaker import ModelPackage
from sagemaker.model_monitor import DataCaptureConfig

model_package_group = 'sagemaker-e2e-d96a4b92-p-yq0atmlyfh0a'
model_version_arn = f"arn:aws:sagemaker:us-east-2:{account_id}:model-package/{model_package_group}/0"

modelmp = ModelPackage(
    role=role,
    model_package_arn=model_version_arn,
    sagemaker_session=sagemaker_session
)

In [ ]:
bucket = sagemaker_session.default_bucket()
prefix = "sm-end-to-end-d96a4b92/demo-model-monitor"

data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)
reports_prefix = "{}/reports".format(prefix)
s3_report_path = "s3://{}/{}".format(bucket, reports_prefix)

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))

In [ ]:
# Deploy the model

endpoint_name = 'demo-e2e-model-monitor-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint Name : ", endpoint_name)

data_capture_config = DataCaptureConfig(enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path)
predictor = modelmp.deploy(initial_instance_count=1, instance_type='ml.m5.large', endpoint_name=endpoint_name, data_capture_config=data_capture_config)